<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R - Coding Bootcamp, Part 3</h1>

This material is closely and gratefully adapted from the work of the UC Berkeley EEP/IAS 118 team, including Jeremy Magruder, Sofia Villas-Boas, James Sears, and many other people working on these materials for EEP118. This is your work. We are in your debt!


## stargazer, xtable, robust standard errors, and fixed effects regressions



This bootcamp will help us understand **stargazer** and **xtable** to produce clear tables of results and summary statistics. We will also explore "robust" standard errors and fixed effects estimates.


<h5>DISCLAIMER:</h5> 
This notebook helps you code clean tables, if that's your thing; it gets technical about robust standard errors that require some extra code to estimate; and it walks you through fixed effects estimation. Of these three things, only the last one is likely the most central to your ECON 140R journey. And be advised that our motivation for fixed effects <i>as they may reveal causal effects under some assumptions</i> is quite different from the more generic motivation presented in the example here. This bootcamp is best viewed as very useful practice, and not as specifics that you are necessarily required to dutifully cover in your own work. In your work, use the tool that works, and that you understand.


Let's load a few packages and read in a dataset on residential water use for residents in Alameda and Contra Costa Counties, `wateruse.dta`. The dataset contains a subset of households who are high water users, people who used over 1,000 gallons per billing cycle. We have information on their water use, weather during the period, as well as information on the city and zip code of where the home is located, and information on the size and value of the house.

In [ ]:
library(tidyverse)
library(haven)
# datahub appears not to have this package installed, so we'll need to install it the first time:
install.packages("lfe")
library(lfe)
library(stargazer)
library(xtable)
library(lmtest)
library(sandwich)

# load in wateruse data
# AND add in measure of gallons per day "gpd" using the variable "unit", which is in CCFs, 100s of cubic feet.
# The conversion factor is 748 gallons per 100 cubic feet. English system shenanigans!
waterdata <- read_dta("wateruse.dta") %>%
    mutate(gpd = (unit*748)/num_days)
head(waterdata)

# Summary Statistics Tables with xtable

`xtable` is a useful package for producing custom summary statistics tables. Let's take it for a spin.

To begin, let's generate a new data frame `homesize` with a subset of data we want, plus some new indicators. We'll select the household identifier `hh`, along with `billincycle`, `gpd`, `degree_days`, and `lotsize`. Next we'll use pipes (1) to drop the missing observations with `drop_na()`, and (2) to create two new categorical variables that separate the data by whether a lot either is less than or greater than one acre (`lotsize_1`), or whether it is less than or greater than 4 acres (`lotsize_4`). Note these definitions are partially overlapping. 

`homesize <- waterdata %>%
    select(hh, billingcycle, gpd, degree_days, lotsize) %>%
    drop_na() %>%
    mutate(lotsize_1 = ifelse((lotsize < 1), "< 1", ">= 1"),
           lotsize_4 = ifelse((lotsize > 4), "> 4", "<= 4"))
head(homesize)`


We can use `summarize()` to create summary statistics for `gpd` and `degree_days` across both levels of lot size, and to assign them to new objects `sumstat_1` and `sumstat_2`. We'll use our `homesize` data frame:

`sumstat_1 <- homesize %>%
    group_by(lotsize_1) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_1`

`sumstat_4 <- homesize %>%
    group_by(lotsize_4) %>%
    summarise(mean_gpd = mean(gpd), 
              mean_degdays = mean(degree_days))
sumstat_4`

And now we can use `xtable()` to put `sumstat_1` and `sumstat_4` into the same table, using its nested options `cbind()` and `t()`. We can also add row names and column names using the code below. Note how we add just the first row to `rownames()`and let __R__ toss in the variable names automatically.

`full <- xtable(cbind(t(sumstat_1), t(sumstat_4)))
    rownames(full)[1] <- "Lotsize Group"
    colnames(full) <- c("lotsize_1 = 1", "lotsize_1 = 0", "lotsize_4 = 0", "lotsize_4 =1")
    full`

We now have a table, `full`, that is an xtable object. Some folks like to use HTML or $\LaTeX$ when they write, and with an xtable object, we can generate those formats using the `print.xtable()` function on our xtable `full`, for example specifying `type = "html":

`print.xtable(full, type = "html")`

Copy and paste the html code into the markdown field below and hit run to see how it appears:

# Regression Tables in Stargazer

`stargazer` is a super useful package for producing professional-quality regression tables. While it defaults to producing $\LaTeX$ formatted tables, a typesetting language that a lot of economists use, we can also produce html code that can easily be copied into markdown cells and run.

If we run the following three regressions: 

\begin{align} 
GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} 
\label{eq1} \tag{1}\\
GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i} 
\label{eq2} \tag{2}\\
GPD_{it} &= \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} + \beta_3 lotsize_{i} + \beta_4 homeval_i \label{eq3} \tag{3}
\end{align}


We might want to present the results side by side in the same table so that we can easily compare coefficients from one column to the other. To do that with `stargazer`, we can 

1. Run each regression, storing them in memory as new objects by using the "gets" operator `<-` 
2. Run `stargazer(reg1, reg2, reg3, ..., type )` where the first arguments are all the regression objects we want in the table, and telling __R__ what type of output we want

If we specify `type = "text"`, we'll get the table displayed directly in the output window:

`reg_a <- lm(gpd ~ degree_days + precip, waterdata)`

`reg_b <- lm(gpd ~ degree_days + precip + lotsize, waterdata)`

`reg_c <- lm(gpd ~ degree_days + precip + lotsize + homeval, waterdata)`

`stargazer(reg_a, reg_b, reg_c, type = "text")`

And if we specify `type = "html"`, we'll get HTML code that we need to copy and paste into a markdown cell:

`stargazer(reg_a, reg_b, reg_c, type = "html")`

Now all we need to do is copy and paste that HTML code from the output into a text cell and we've got our table!

copy your HTML code here and run it

And we get a nice looking regression table with all three models side by side. This makes it easy to see how the coefficient on lot size falls when we add in home value, letting us quickly figure out the (rather obvious) sign of correlation between lot size and home value.

## Table Options

Stargazer has many different options for customizing the look of our table with optional arguments, including
* `title` lets us add a custom title
* `column.labels` lets you add text labels to the columns
* `covariate.labels` lets us specify custom labels for all our variables other than the variable names. Specify each label in quotations in the form of a vector with `c()`
* `ci = TRUE` adds in confidence intervals (by default for the 10\% level, but you can change it to the 1\% level with `ci.level = 0.99`
* `intercept.bottom = FALSE` will move the constant to the top of the table
* `digits` lets you choose the number of decimal places to display
* `notes` lets you add some notes at the bottom

For example, we could customize the above table as

`stargazer(reg_a, reg_b, reg_c, type = "text",
         title = "Water Use, Weather, and Home Characteristics",
         column.labels = c("Weather", "With Lotsize", "With HomeVal"),
          covariate.labels = c("Intercept", "Degree Days", "Precipitation (mm)", 
              "Lot Size (Acres)", "Home Value (USD)"),
          intercept.bottom = FALSE,
          digits = 2,
          note = "Isn't stargazer neat?"
          )`

# Summary Statistics Tables in Stargazer

Stargazer is also useful if we have a lot of variables we'd like to summarize, perhaps with other "group" variables we'd like to compare them across. This approach works especially well with `across()` within `summarize()`.

For example, suppose we wanted to summarize the median and variance of `gpd`, `precip`, and `degree_days` by whether the home was built after 1980 or not. Rather than create separate tables for all of the variables and merge them together like with xtable, we can summarize across using clever calls to `mutate()` with `ifelse()` and pipes:

`ss_acr <- mutate(waterdata, pre_80 = ifelse(yearbuilt < 1980, "1. Pre-1980", "2. 1980+")) %>%
    group_by(pre_80) %>%
    summarize(across(.cols = c(gpd, precip, degree_days),
                     .fns = list(Median = median, Variance = var)))
ss_acr`

Note that `ifelse()` is a function that follows the format

`ifelse(Condition, Value if True, Value if False)`

Here our condition is that the $yearbuilt$ variable is less than 1980. If it’s true, we want this
new variable to take on the label "1. Pre-1980", and otherwise
be "2. 1980+".

This table then contains everything we want, but having it displayed "wide" like this is a bit tough to see. If we wanted to display it "long" where there is one column for each or pre-1980 and post-1980 homes, we can just use the transpose function `t()` applied to the object we just created. We can place that within a `stargazer()` call and ask for html code:

`stargazer(t(ss_acr), type = "html")`

(copy your html code here)

## Heteroskedasticity-Robust Standard Errors 


We might often like to report standard errors that are robust to certain typical forms of <i>heteroskedasticity</i>, where subgroups have their own variances. Many economists just report heteroskedasticity-robust standard errors by default, even. It isn't a magic fix for all problems, but it's often good practice.

To add robust standard errors to our table, we'll take advantage of the `lmtest` and `sandwich` packages. We loaded these in the preamble. The Wikipedia page on [heteroskedasticity-consistent (HC) standard errors](https://en.wikipedia.org/wiki/Heteroskedasticity-consistent_standard_errors) provides an overview if you're interested, and it likely explains where the `HC1` comes from below.

If we want to see the coefficient table from Regression B with "robust" standard errors, by which we mean corrected for heteroskedasticity and often called Huber-White standard errors, we can use the `coeftest()` function as specified below:

`coeftest(reg_b, vcov = vcovHC(reg_b, type = "HC1"))`

The `vcovHC(reg_b, type = "HC1")` argument tells __R__ that we want to calculate standard errors using the heteroskedasticity-robust approach, which means functionally that we want a specific form of the variance-covariance matrix between our residuals. `coeftest()` then prints the nice output table. 

While this is a perfectly reasonable way to view the robust standard errors in a summary-style table, we can also extract the robust standard errors so we can insert them elsewhere, like into stargazer.

To retrieve a vector of robust standard errors from Regression B, we can: 
* call `vocHC(reg_b, type = "HC1")`
* pass it to `diag()` in order to retrieve just the estimated variances and not the covariances
* take the squart root with `sqrt()`

`robust_b <- sqrt(diag(vcovHC(reg_b, type = "HC1")))`

`robust_b`

Let's break this call down step-by-step to understand it better.

`vcov_b <- vcovHC(reg_b, type = "HC1")`

This first `vcov_b` object is getting the entire variance-covariance matrix for our regression coefficients. Since we again specified `type = "HC1"`, we ensure we get the heteroskedasticity-robust version of this matrix. If we had instead specified `type = "constant"` we would be assuming homoskedasticity and would get our usual variance estimates.

What this looks like is


$$VCOV_b = \begin{matrix}{}
\widehat{Var}(\hat \beta_0)       & \widehat{Cov}(\hat \beta_0, \hat\beta_1)  & \widehat{Cov}(\hat \beta_0, \hat\beta_2)  \\
 \widehat{Cov}(\hat \beta_1, \hat\beta_0) & \widehat{Var}(\hat \beta_1)         & \widehat{Cov}(\hat \beta_1, \hat\beta_2) \\
 \widehat{Cov}(\hat \beta_2, \hat\beta_0)  & \widehat{Cov}(\hat \beta_2, \hat\beta_1) & \widehat{Var}(\hat \beta_2)  
\end{matrix}$$

Where each element is $\hat\sigma_i$ in the ith row mutiplied by $\hat\sigma_j$ in the jth column. Note that when $i = j$ in the main diagonal, we get the variance estimate for $\hat \beta_i$.

You can check this by running the following lines:

`vcov_b <- vcovHC(reg_b, type = "HC1")
 vcov_b`

`var_b <- diag(vcov_b)`

The `diag()` function extracts this main diagonal, giving us a vector of our robust estimated variances

`robust_b <- sqrt(var_b)`

And taking the square root gets us our standard error estimates for our $\hat\beta$'s. See the process by running the following lines:

`var_b <- diag(vcov_b)`

`var_b`

`robust_b <- sqrt(var_b)`

`robust_b`

## Stargazer and Heteroskedasticity-Robust Standard Errors 

Now that we know how to get our robust standard errors, we can construct them for all three of our regressions and add them to our stargazer table:

`robust_a <- sqrt(diag(vcovHC(reg_a, type = "HC1")))`

`robust_b <- sqrt(diag(vcovHC(reg_b, type = "HC1")))`

`robust_c <- sqrt(diag(vcovHC(reg_c, type = "HC1")))`


`stargazer(reg_a, reg_b, reg_c, 
           type = "html",
           se = list(robust_a, robust_b, robust_c),
           omit.stat = "f")`
          
Here we're adding the robust standard errors to `stargazer()` with the `se =` argument, combining them together <u>in the right order</u> as a list. We also ideally omit the overall F test at the bottom with `omit.stat = "f"`, because we would have needed to correct that for heteroskedasticity also. Perhaps another day. 

Try running this code below to see how the standard errors change when we use robust standard errors:

Copy and paste the table code below and run the cell to see it formatted.


# Fixed Effects Regression


"Fixed Effets" is a broad term that refers to many model specifications, with the common theme that some estimated effects are constrained to be fixed. If we have panel data, where units of observation like people, firms, or households are observed repeatedly, we can model "time fixed effects," meaning an effect of a time period that is common across units. We can also model "individual fixed effects," which are constant over time for individual units. A step short of individual fixed effects would be geographic fixed effects, for example states or cities which many individuals in the dataset have in common.

Let's give this a try using the dataset `wateruse.dta`. 

Suppose we were interested in running the following panel regression of residential water use:

$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it} ~~~~~~~~~~~~~~~~~~~~~~~(1)$$

Where $GPD_{it}$ measures the gallons used per day by household $i$ in billing cycle $t$, $degree\_days$ is the count of degree days* experienced by the household in that billing cycle, and $precip$ is the amount of precipitation in millimeters.

\* (Degree days are a measure of cumulative time spent above a certain temperature threshold.)

We begin our analysis in the usual way: we assume that $\beta_0$ is a constant and also the same <i>across all units of analysis</i>, here indexed by $i$, and also the same across units of time.

`reg1 <- lm(gpd ~ degree_days + precip, data = waterdata)`

`summary(reg1)`

Here we obtain an estimate of $\hat\beta_1 = 0.777$, telling us that an additional degree day per billing cycle is associated with an additional $0.7769$ gallon used per day. These billing cycles are roughly two months long, so this suggests an increase of roughly 47 gallons per billing cycle. Our estimate is statistically significant at all conventional levels, suggesting residential water use does respond to increased exposure to high heat.

We estimate a statistically insignificant coefficient on additional precipitation, $\hat{\beta}_2 = 0.4572$, which tells us that on average, household water use in our sample doesn't adjust to how much it rains.

We might think that characteristics of the home impact how much water is used there, so let's add in some controls for home characteristics:


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \beta_3 lotsize_{i} + \beta_4 homesize_i + \beta_5 num\_baths_i + \beta_6 num\_beds_i + \beta_7 homeval_i~~~~~~~~~~~~~~~~~~~~~~~(2)$$

`reg2 <- lm(gpd ~ degree_days + precip + lotsize + homesize + num_baths + num_beds + homeval, data = waterdata)
summary(reg2)`

In this longer regression, our coefficient on `degree_days`, $\hat\beta_1$, remains statistically significant and doesn't change much. Another way to describe the story thus far is that we find that $\hat\beta_1$ is robust to the addition of home characteristics. Perhaps this isn't surprising; `degree_days` is a climatic variable, after all.

Of these characteristics, we obtain statistically significant coefficients on the size of the lot in acres (`lotsize`), the size of the home in square feet (`homesize`), and the number of bedrooms in the home (`num_beds`).

We get a curious result for $\hat\beta_6$: for each additional bedroom in the home we predict that water use will *fall* by 48 gallons per day, shown in the regression table as $\hat{\beta}_6 = -4.791 \times 10^1$.  

### Motivation for fixed effects: If you can, you should, because it's worth checking

If we have panel data with repeated observations of units, then what we just ran, usually called "pooled OLS" because we ran OLS on a big pool of repeated observations of units, might not be ideal. To see why in a simple way, just consider how the error terms $\epsilon_{it}$ for one unit $i$ might deviate systematically from zero over time, but they might be accidentally balanced out by errors for another unit that deviate in the other direction. Neither of those is what we want; the error term should be white noise.

Since there are likely a number of sources of omitted variable bias in the previous model, we think it might be worth including some fixed effects in our model. These will allow us to control for some of the unobserved sources of OVB without needing to measure them directly. Modeling a <i>fixed effect</i>, so called because it does not vary, would sweep out any impacts of omitted variables that do not change over time.

But here are two big caveats:
* Fixed effects can wash out a lot of identifying variation in the data. In some cases, that might be perfectly acceptable. But in others, it might be like using a hacksaw to slide an almond: way too powerful of a tool to leave anything other than dust in its wake, even where there should be something left.

* Especially in sociology and other disciplines, another related estimation method called <i>random effects</i> is a common approach for panel data. A benefit is that random effect do not wash out all the effects of unchanging covariates, like fixed effects do. Perhaps we'll study random effects another time.

## Method 1: Fixed Effects with lm() 

We can write a general panel fixed effects model as 

$$ y_{it} = \beta x_{it} + \mathbf{a}_i + \mathbf{d}_t + u_{it} $$

* $y$ our outcome of interest, which varies in both the time and cross-sectional dimensions
* $x_{it}$ our set of time-varying unit characteristics
* $\mathbf{a}_i$ our set of unit fixed effects
* $\mathbf{d}_t$ our time fixed effects

We can estimate this model in `lm()` provided we have variables in our dataframe that correspond to each level of $a_i$ and $d_t$. This often means we'll have to generate them before we can run the regression, and this can be computationally intensive if there are lots of fixed effects.

We can tell __R__ to create fixed effects for variables in an `lm()` regression by using a call to `factor()`. Suppose we wanted to specify fixed effects for cities in our data, where cities are measured in `city`. 

`reg3 <- lm(gpd ~ degree_days + precip + factor(city), data = waterdata)
summary(reg3)`

Note that there is a `city == 1`, and __R__ has dropped it by default and kept the intercept. In case like this, that city "gets" the constant term, while `city == 2` gets the constant plus its fixed effect. (Households in both cities also get the other effects of the other right-hand side variables too, of course).

Suppose we'd like to run this regression:


$$ GPD_{it} = \beta_0 + \beta_1 degree\_days_{it} + \beta_2 precip_{it}  + \mathbf{a}_i + \mathbf{d}_t~~~~~~~~~~~~~~~~~~~~~~~(2)$$

where $\mathbf{a}_i$ are our city fixed effects, and $\mathbf{d}_t$ are billing cycle (time or season) fixed effects:

`fe_reg1 <- lm(gpd ~ degree_days + precip + factor(city) + factor(billingcycle), data = waterdata)
summary(fe_reg1)`

__R__ automatically chose the first dummy variable for each set of fixed effect (city 1 and billing cycle 1) to leave out as our omitted group. 

Now that we've accounted for which billing cycle we're in (i.e., whether we're in the winter or the summer), we find that the coefficient on `degree_days` is now much smaller and statistically insignificant. We can and should reason through this result: 

In the model without seasonal or time fixed effects, any extra water use that might have been associated with seasonality (an omitted variable there) would probably have been attributed to temperature and partially captured by the coefficient on `degree days`. Now that we are controlling for the season via `billingcycle` fixed effects, we find that deviations in temperature exposure during a billing cycle don't result in dramatically higher water use within the sample.

## Method 2: Fixed Effects with felm()

Alternatively, we could do everything faster using the `felm()` function from the package __lfe__. This package performs the background math faster, so it will estimate fixed effects models using large datasets and many variables more efficiently.

The syntax we use is now 

`felm(y ~ x1 + x2 + ... + xk | FE_1 + FE_2 + ..., data = df)`

* The first section $y \sim x1 + x2 +... xk$ is our formula, written the same way as with `lm()` - but omitting the fixed effects
* We now add a `|` and in the second section we specify our fixed effects. Here we say $FE\_1 + FE\_2$ which tells __R__ to include fixed effects for each level of the variables $FE\_1$ and $FE\_2$.
* we add the data source after the comma, as before.

Let's go ahead and try this now with our water data model:

`fe_reg2 <- felm(gpd ~ degree_days + precip | city + billingcycle, data = waterdata)
summary(fe_reg2)`

`felm()` ran more quickly, and it produces the same estimates of the coefficients on `degree_days` and `precip`. We didn't have to mutate our data or add any variables. A potential downside is that this approach doesn't report the fixed effects themselves by default. But `felm` runs a lot faster than `lm`, especially with large datasets. 

We can also recover the fixed effects with getfe(): 

`getfe(fe_reg2, se = TRUE, robust = TRUE)`

the argument `se = TRUE` tells it to produce standard errors too, and `robust = TRUE` further indicates that we want heteroskedasticity-robust standard errors.


Note that this approach doesn't give you the same reference groups as before, but we get the same relative values. Note that before the coefficient on `city.2` was 301.7 and now is 73.9. But the coefficient on `city.1` $= -227.8$, and if we subtract `city.1` from `city.2` to get the difference in averages for city 2 relative to city 1 we get $73.9 - (-227.8) = 301.7$, the same as before.

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>